In [ ]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map.add_basemap('Google Satellite')
Map

In [ ]:
roi = Map.draw_last_feature
print(roi.geometry().getInfo())

In [ ]:
class landsat_ndvi_seasonality(object):
                               
    def __init__(self, roi=None, start_year=2014, end_year=2018):
        
        '''Start the instance with the parameters for start and end year. Also we set the fixed values for
        the seasons dates and generate the collection as ndvi 32 days merge of all landsats'''
        
        
        #year = datetime.datetime.now().year

        #ee_initialize()
        self.roi = roi
        if self.roi is None:
            self.roi = ee.Geometry.Polygon(
                [[[-6.766047, 36.776586], 
                  [-6.766047, 37.202186], 
                  [-5.867729, 37.202186], 
                  [-5.867729, 36.776586], 
                  [-6.766047, 36.776586]]], None, False)
            
        else:

            if not isinstance(roi, ee.Geometry):

                try:
                    self.roi = self.roi.geometry()
                except Exception as e:
                    print('Could not convert the provided roi to ee.Geometry')
                    print(e)
                    return

                
        self.start_year = start_year
        self.end_year = end_year
        self.winter = ['-01-01', '-03-31']
        self.spring = ['-04-01', '-06-30']
        self.summer = ['-07-01', '-09-30']
        self.autumn = ['-10-01', '-12-31']

        # Landsat collection preprocessingEnabled
        # Get Landsat surface reflectance collections for OLI, ETM+ and TM sensors.
        LC08col = ee.ImageCollection("LANDSAT/LC08/C01/T1_8DAY_NDVI")
        LE07col = ee.ImageCollection("LANDSAT/LE07/C01/T1_8DAY_NDVI")
        LT05col = ee.ImageCollection("LANDSAT/LT05/C01/T1_8DAY_NDVI")
        LT04col = ee.ImageCollection("LANDSAT/LT04/C01/T1_8DAY_NDVI")

        self.ndvi_col = LC08col.merge(LE07col).merge(LT05col).merge(LT04col)
        #self.ndvi_col = LC08col.merge(LT05col).merge(LT04col)
        self.imagelist = []
    
    def get_winter(self, y):
        
        '''Here comes the funny thing. We have to generate the winter image for each year'''
        init, ends = str(y) + self.winter[0], str(y) + self.winter[1]
        return self.ndvi_col.filterDate(init, ends).select('NDVI').max()
        
    def get_spring(self, y):
        
        '''Here comes the funny thing. We have to generate the winter image for each year'''
        init, ends = str(y) + self.spring[0], str(y) + self.spring[1]
        return self.ndvi_col.filterDate(init, ends).select('NDVI').max()
        
    def get_summer(self, y):
        
        '''Here comes the funny thing. We have to generate the winter image for each year'''
        init, ends = str(y) + self.summer[0], str(y) + self.summer[1]
        return self.ndvi_col.filterDate(init, ends).select('NDVI').max()
        
    def get_autumn(self, y):
        
        '''Here comes the funny thing. We have to generate the winter image for each year'''
        init, ends = str(y) + self.autumn[0], str(y) + self.autumn[1]
        return self.ndvi_col.filterDate(init, ends).select('NDVI').max()
    
    def get_year_composite(self):
        
        '''return the composite ndvi for each year'''
        for y in range(self.start_year, self.end_year):
            
            composite = ee.Image.cat(self.get_winter(y), self.get_spring(y), self.get_summer(y), self.get_autumn(y)).clip(self.roi)

            compositer = composite.select(['NDVI', 'NDVI_1', 'NDVI_2', 'NDVI_3'], ['invierno', 'primavera', 'verano', 'autumn'])
            #self.col = ee.ImageCollection(compositer)
            self.imagelist.append(compositer)
        
        print(len(self.imagelist))
        ndvi_comp_coll = ee.ImageCollection.fromImages(self.imagelist)
        
        return ndvi_comp_coll


In [ ]:
a = landsat_ndvi_seasonality(roi, 1995, 2019)

In [ ]:
b2010 = a.get_year_composite()

In [ ]:
Map.addLayer(b2010, {'bands': ['invierno', 'primavera', 'verano'], 'min': 0.1, 'max': 0.8})
#Map.addLayer(b2010, {'bands': ['primavera', 'verano', 'autumn'], 'min': 0.2, 'max': 0.75})
#Map.addLayer(b2010)

In [ ]:
# Now I use the wonderful geemap library to create the gif
# Define arguments for animation function parameters.
video_args = {
  'dimensions': 768,
  'region': roi, #.geometry().getInfo()['coordinates'],
  'framesPerSecond': 1,
  'bands': ['invierno', 'primavera', 'verano'], #Yes, I should change band names to winter, spring and summer
  'min': 0.1,
  'max': 0.8,
  'gamma': [1, 1, 1],
  'maxPixels': 1e15
}

In [ ]:
work_dir = os.path.join('/home/diego/Descargas')
out_gif = os.path.join(work_dir, "ndvi_multi_class_dessert_10.gif")

In [ ]:
geemap.download_ee_video(a.get_year_composite(), video_args, out_gif)

In [ ]:
texted_gif = os.path.join(work_dir, "ndvi_Musa_text.gif")
geemap.add_text_to_gif(out_gif, texted_gif, xy=('90%', '2%'), text_sequence='1995', font_size=30, font_color='#ffffff', add_progress_bar=False)

In [ ]:
geemap.show_image(texted_gif)

In [ ]:
print(a.get_year_composite().size())